In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, Normalizer
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType, StringType, IntegerType
import json
from pyspark.ml.feature import OneHotEncoder, StringIndexer

In [3]:
conf = SparkConf()
conf.set("spark.app.name", "evgeniy osipchuk lab03") 

spark = SparkSession.builder.config(conf=conf).appName("evgeniy osipchuk lab03").getOrCreate()

### Считывание данных

Для более красивого вывода переопределим функцию show так, чтобы данные выводились в формате pandas. Pandas будет использоваться только для вывода на экран

In [4]:
import pyspark

def show(self, n=5):
    return self.limit(n).toPandas()

pyspark.sql.dataframe.DataFrame.show = show

In [5]:
!hdfs dfs -ls /labs/slaba03/

Found 4 items
-rw-r--r--   3 hdfs hdfs   91066524 2021-02-27 22:12 /labs/slaba03/laba03_items.csv
-rw-r--r--   3 hdfs hdfs   29965581 2021-02-27 22:12 /labs/slaba03/laba03_test.csv
-rw-r--r--   3 hdfs hdfs   74949368 2021-02-27 22:12 /labs/slaba03/laba03_train.csv
-rw-r--r--   3 hdfs hdfs  871302535 2021-02-27 22:12 /labs/slaba03/laba03_views_programmes.csv


In [6]:
items = spark.read.load("/labs/slaba03/laba03_items.csv",
                        format = "csv",
                        sep="\t",
                        header=True,
                        inferSchema=True)

In [7]:
test = spark.read.load("/labs/slaba03/laba03_test.csv",
                        format = "csv",
                        sep=",",
                        header=True,
                        inferSchema=True)
train = spark.read.load("/labs/slaba03/laba03_train.csv",
                        format = "csv",
                        sep=",",
                        header=True,
                        inferSchema=True)
views_programmes = spark.read.load("/labs/slaba03/laba03_views_programmes.csv",
                        format = "csv",
                        sep=",",
                        header=True,
                        inferSchema=True)

In [8]:
items.show(5)

,item_id,channel_id,datetime_availability_start,datetime_availability_stop,datetime_show_start,datetime_show_stop,content_type,title,year,genres,region_id
0,7006336,9.0,1970-01-01 03:00:00,2017-04-04 12:00:00,2017-04-01 11:05:00,2017-04-01 12:00:00,0,крылатый властелин морей,NaN,General,41.0
1,7006349,9.0,1970-01-01 03:00:00,2017-04-04 23:55:00,2017-04-01 23:00:00,2017-04-01 23:55:00,0,крылатый властелин морей,NaN,General,41.0
2,7006348,9.0,1970-01-01 03:00:00,2017-04-04 23:00:00,2017-04-01 21:10:00,2017-04-01 23:00:00,0,билокси-блюз,NaN,General,41.0
3,7006341,9.0,1970-01-01 03:00:00,2017-04-04 15:30:00,2017-04-01 15:00:00,2017-04-01 15:30:00,0,новости культуры с владиславом флярковским,NaN,General,41.0
4,7006342,9.0,1970-01-01 03:00:00,2017-04-04 16:10:00,2017-04-01 15:30:00,2017-04-01 16:10:00,0,предки наших предков,NaN,General,41.0


In [9]:
test.show(2)

,user_id,item_id,purchase
0,1654,94814,None
1,1654,93629,None


In [10]:
train.show(2)

,user_id,item_id,purchase
0,865692,102141,0
1,865692,9593,0


In [11]:
views_programmes.show(2)

,user_id,item_id,ts_start,ts_end,item_type
0,0,7101053,1491409931,1491411600,live
1,0,7101054,1491412481,1491451571,live


In [12]:
train.count()

5032624

In [13]:
train.select('user_id').distinct().count()

1941

In [14]:
test.count()

2156840

In [15]:
test.select('user_id').distinct().count()

1941

Сосредоточьтесь на формировании следующих фичей: по файлу laba03_train.csv сформируйте признаки, характеризирующие как интенсивно покупает пользователь и "покупаемость" item'ов  
выбросить лишние фичи  
Оконной функцией сгруппировать по пользователю и по итему отдельно. Подсчитать долю покупок.  
Сделать мин-макс преобразование всех фичей.   

Выбросим лишние признаки

In [16]:
items = items.select(['item_id', 'channel_id', 'content_type', 'year', 'genres', 'region_id'])

Заполним пропуски 

In [17]:
items = items.fillna('unknown', subset=['genres'])

In [18]:
items = items.fillna(0)

Выделим все жанры в отдельные признаки

In [19]:
genres_list = []
for item in items.select('genres').distinct().collect():
    for genre in item.genres.split(','):
        genres_list.append(genre)    
genres_list = set(genres_list)

In [20]:
def custom_ohe_udf(genre):
    return F.udf(lambda col: 1 if genre in col else 0, IntegerType())

In [21]:
for genre in genres_list:
    items = items.withColumn(genre, custom_ohe_udf(genre)(F.col("genres")))

Выбросим неактуальную колонку genres

In [22]:
items = items.drop('genres')
items.cache()

DataFrame[item_id: int, channel_id: double, content_type: int, year: double, region_id: double, Комедии: int, Детские песни: int, Мультсериалы: int, Триллер: int, Детективы: int, Союзмультфильм: int, Фильмы: int, Сериалы: int, Для детей: int, Мистические: int, Мультфильмы: int, Для всей семьи: int,  сказка: int, Юмористические: int, Передачи: int, Западные мультфильмы: int, Драма: int, Советское кино: int, Мультфильм: int, Короткометражки: int, Приключение: int, Научная фантастика: int, Военный: int, Музыкальные: int, Реалити-шоу: int, Хочу всё знать: int, Ужасы: int, Полнометражные: int, Фантастика: int, Советские: int, Русские мультфильмы: int, Фильмы-спектакли: int, Мюзиклы: int, Мелодрама: int, Документальные: int, Сказки: int, Спортивные: int, Артхаус: int, Кулинария: int, Фильмы в 3D: int, Комедия: int, Документальный: int, Познавательные: int, Приключения: int, Видеоигры: int, Экранизации: int, Русские: int, Вестерн: int, Фэнтези: int, unknown: int, Развлекательные: int, Анимаци

In [23]:
items.show()

,item_id,channel_id,content_type,year,region_id,Комедии,Детские песни,Мультсериалы,Триллер,Детективы,...,Военные,Музыкальный,Боевики,Для самых маленьких,Детские,Про животных,Спорт,Романтические,Боевик,Прочие
0,7006336,9.0,0,0.0,41.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7006349,9.0,0,0.0,41.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7006348,9.0,0,0.0,41.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7006341,9.0,0,0.0,41.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7006342,9.0,0,0.0,41.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Объединим train и test c items

In [24]:
train_joined = train.join(items, on='item_id', how='left')

In [25]:
test_joined = test.join(items, on='item_id', how='left')

Для каждого пользователя определим количество и долю его покупок, а для каждого фильма - долю его покупаемости

In [26]:
from pyspark.sql.window import Window
w = Window.partitionBy('user_id')
w_item = Window.partitionBy('item_id')

In [27]:
train_joined = (train_joined
                .withColumn('user_buy_part', F.sum('purchase').over(w) / F.count('user_id').over(w))
                .withColumn('item_buy_part', F.sum('purchase').over(w_item) / F.count('user_id').over(w_item))
                .withColumn('user_buy_count', F.sum('purchase').over(w)))
train_joined.cache()

DataFrame[item_id: int, user_id: int, purchase: int, channel_id: double, content_type: int, year: double, region_id: double, Комедии: int, Детские песни: int, Мультсериалы: int, Триллер: int, Детективы: int, Союзмультфильм: int, Фильмы: int, Сериалы: int, Для детей: int, Мистические: int, Мультфильмы: int, Для всей семьи: int,  сказка: int, Юмористические: int, Передачи: int, Западные мультфильмы: int, Драма: int, Советское кино: int, Мультфильм: int, Короткометражки: int, Приключение: int, Научная фантастика: int, Военный: int, Музыкальные: int, Реалити-шоу: int, Хочу всё знать: int, Ужасы: int, Полнометражные: int, Фантастика: int, Советские: int, Русские мультфильмы: int, Фильмы-спектакли: int, Мюзиклы: int, Мелодрама: int, Документальные: int, Сказки: int, Спортивные: int, Артхаус: int, Кулинария: int, Фильмы в 3D: int, Комедия: int, Документальный: int, Познавательные: int, Приключения: int, Видеоигры: int, Экранизации: int, Русские: int, Вестерн: int, Фэнтези: int, unknown: int, 

In [28]:
train_joined.show()

,item_id,user_id,purchase,channel_id,content_type,year,region_id,Комедии,Детские песни,Мультсериалы,...,Для самых маленьких,Детские,Про животных,Спорт,Романтические,Боевик,Прочие,user_buy_part,item_buy_part,user_buy_count
0,5117,566758,0,0.0,1,2011.0,0.0,0,0,0,...,0,0,0,0,0,1,0,0.000377,0.001409,1
1,8257,566758,0,0.0,1,1974.0,0.0,0,0,0,...,0,1,0,0,0,0,0,0.000377,0.031433,1
2,8407,566758,0,0.0,1,2008.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0.000377,0.001468,1
3,11025,566758,0,0.0,1,2016.0,0.0,1,0,0,...,0,0,0,0,0,1,0,0.000377,0.000000,1
4,71600,566758,0,0.0,1,2005.0,0.0,0,0,0,...,0,0,0,0,1,0,0,0.000377,0.000000,1


Для корректного предсказания в test пользователям и фильмам присвоим соответствующие значения сгенерированных признаков из train

In [29]:
train_grouped_user_id = (train_joined
                 .select('user_id', 'user_buy_part', 'user_buy_count')
                 .groupBy('user_id')
                 .agg(F.mean('user_buy_part').alias('user_buy_part'),
                      F.mean('user_buy_count').alias('user_buy_count')))
train_grouped_item_id = (train_joined
                 .select('item_id', 'item_buy_part',)
                 .groupBy('item_id')
                 .agg(F.mean('item_buy_part').alias('item_buy_part')))

In [30]:
train_grouped_user_id.show()

,user_id,user_buy_part,user_buy_count
0,793876,0.001940,5.0
1,795620,0.004244,11.0
2,822709,0.000379,1.0
3,824008,0.000382,1.0
4,851848,0.000389,1.0


In [31]:
test_joined = (test_joined
               .join(train_grouped_user_id, on='user_id', how='left')
               .join(train_grouped_item_id, on='item_id', how='left'))

In [31]:
#train_joined = train_joined.filter(F.col('content_type') == 1)

### Обучение

Проведем векторизацию признаков

In [32]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

In [33]:
train_columns = train_joined.columns

In [34]:
train_columns = list(set(train_columns) - set(['purchase',]))

In [35]:
assembler = VectorAssembler(inputCols=train_columns,
                            outputCol='features')
output_train = assembler.transform(train_joined)

In [36]:
lr = LogisticRegression(featuresCol='features', labelCol="purchase")

In [37]:
lr_model = lr.fit(output_train)

In [38]:
output_test = assembler.transform(test_joined)

In [39]:
predictions = lr_model.transform(output_test)

In [40]:
predictions.show()

/usr/hdp/current/spark2-client/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


,item_id,user_id,purchase,channel_id,content_type,year,region_id,Русские,Сериалы,сказка,...,Наши,Музыкальный,Охота и рыбалка,user_buy_part,user_buy_count,item_buy_part,features,rawPrediction,probability,prediction
0,8389,761341,None,0.0,1,1981.0,0.0,0,0,0,...,1,0,0,0.000388,1.0,0.005979,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[6.264997592541692, -6.264997592541692]","[0.9981018916700156, 0.0018981083299843902]",0.0
1,8389,776188,None,0.0,1,1981.0,0.0,0,0,0,...,1,0,0,0.001153,3.0,0.005979,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[6.250032443729611, -6.250032443729611]","[0.9980733277256817, 0.0019266722743182644]",0.0
2,8389,846231,None,0.0,1,1981.0,0.0,0,0,0,...,1,0,0,0.001924,5.0,0.005979,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[6.2716616952746325, -6.2716616952746325]","[0.9981144750335804, 0.0018855249664195544]",0.0
3,8389,765780,None,0.0,1,1981.0,0.0,0,0,0,...,1,0,0,0.001949,5.0,0.005979,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[6.206725866210215, -6.206725866210215]","[0.9979882286249971, 0.0020117713750028706]",0.0
4,8389,816244,None,0.0,1,1981.0,0.0,0,0,0,...,1,0,0,0.001923,5.0,0.005979,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[6.250851628237683, -6.250851628237683]","[0.9980749023423404, 0.001925097657659639]",0.0


In [41]:
def save_to_csv(predictions):
    predictions = predictions.select("user_id", "item_id", "prediction", "probability", "rawPrediction")

    return_probability_udf = F.udf(lambda v: v.values.tolist()[1], DoubleType())

    predictions = predictions.withColumn('purchase', return_probability_udf(F.col('probability'))).sort(F.asc('user_id'), F.asc('item_id'))

    # Почему-то сохранение через pyspark непонятно где создавало файл. Пришлось сохранить через Pandas
    answer = predictions.select("user_id", "item_id", 'purchase').toPandas()

    answer.to_csv('lab03.csv')

In [293]:
save_to_csv(predictions)

In [42]:
from pyspark.ml.classification import GBTClassifier

In [43]:
gbt = GBTClassifier(featuresCol='features', labelCol="purchase")

In [44]:
gbt_model = gbt.fit(output_train)

In [45]:
predictions_gbt = gbt_model.transform(output_test)

In [46]:
save_to_csv(predictions_gbt)